In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

In [ ]:
pythia_fragment_CP5 = """
import FWCore.ParameterSet.Config as cms

from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentGeneratorFilter",
      comEnergy = cms.double(13000.0),
      crossSection = cms.untracked.double(1.095e-3),
      filterEfficiency = cms.untracked.double(1),
      maxEventsToPrint = cms.untracked.int32(0),
      pythiaHepMCVerbosity = cms.untracked.bool(False),
      pythiaPylistVerbosity = cms.untracked.int32(1),
      PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        processParameters = cms.vstring(
        __CHANNEL_DECAY_FRAGMENT__
        ),
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'pythia8PSweightsSettings',
                                    'processParameters',
                                    )
      )
)

ProductionFilterSequence = cms.Sequence(generator)

"""

In [ ]:
def fragmentsDictCreator(decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2016APV': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [ ]:
years = ["2016", "2016APV", "2017", "2018"]
mX = [700, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 3000, 3500]
widths = ["0p014", "5p6", "10p0"]
widths_num = [0.00014, 0.056, 0.1]
tot_events = [900000, 600000, 450000, 420000, 390000, 360000, 360000, 360000, 360000, 360000, 360000, 360000, 360000, 360000]
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_datasetname_year = 'GluGluSpin0ToZGToQQG_M-{mass}_Width{width}_TuneCP5_13TeV-pythia8'
    tmp_dataset_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    
    dataset_names_year = []
    gp_ggf_year = []

    for mx in mX:
        for width in widths:
            d = {"mass": str(mx), "width": width}
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict = dataset_names_year
        
    dataset_names[years[year]] = tmp_dataset_dict

In [ ]:
z_gamma = '''                                                                                                                                                                                           
    'HiggsSM:gg2H = on',
    '25:m0 = __X_MASS__',
    '25:mWidth = __X_WIDTH__',
    '25:onMode = off',
    '25:OnIfMatch = 23 22',
    '25:doForceWidth = on',
    'Higgs:clipWings = off',
    
    '23:onMode = off',
    '23:OnIfMatch = 1 1',
    '23:OnIfMatch = 2 2',
    '23:OnIfMatch = 3 3',
    '23:OnIfMatch = 4 4',
    '23:OnIfMatch = 5 5',
'''

## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator(z_gamma)
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in years:
    #print(year)
    with open('Zgamma' + '_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        '''if year == "2016":
            pr = 5208
        if year == "2016APV":
            pr = 5224
        if year == "2017":
            pr = 4824
        if year == "2018":
            pr = 5152'''
            
        k=0
        for i in range(len(mX)):
            for j in range(len(widths)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.6.5"
                dataset_name = dataset_names[year][k]
                if year == "2016APV":
                    events = round(tot_events[i]*0.54)
                    #prepid = 'B2G-RunIISummer20UL16wmLHEGENAPV-0' + str(pr)
                elif year == "2016":
                    events = round(tot_events[i]*0.46)
                    #prepid = 'B2G-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                elif year == "2017":
                    events = tot_events[i]
                    #prepid = 'B2G-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                elif year == "2018":
                    events = tot_events[i]
                    #prepid = 'B2G-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                
                tmp_fragment = process_pythia_map[year]
                final_fragment = tmp_fragment.replace('__X_MASS__', str(float(mX[i])))
                final_fragment = final_fragment.replace('__X_WIDTH__', "{:.3f}".format(mX[i]*widths_num[j]))
                note = dataset_name.replace('_',' ')
                #generators="Madgraph_" + version + " Pythia8"
                generators="Pythia8"
                mcdb_id = '0'
                time = '0.01'
                size = '30'
                csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                #pr = pr+1   
                k=k+1